**Домашнее задание 1, теоретическая часть.**

**Луничкин Егор, 396 гр.**

# Контрольные вопросы (10%)

**1)** Что является объектом в задаче обучения ранжированию? Какой смысл имеют целевые метки? Какие объекты сравнимы между собой?

Объектом в задаче ранжирования является пара $(q, d)$, где $q~-$ запрос, а $d~-$ документ, то есть мы хотим для каждого запроса возвращать наиболее "релевантный" для него документ.

Целевая метка$~-$ это, соответственно, мера "релевантности" документа $d$ запросу $q$. Сравнимы между собой только документы в рамках одного и того же запроса.

**2)** В чём преимущество метрики NDCG перед метрикой MAP?

В метрике NDCG в отличие от метрики MAP функции оценки релевантности могут быть вещественными, то есть мы можем оценить степень релевантности документа $d$ запросу $q$, а не просто понять бинарный факт: релевантен запрос или нет.

**3)** Опишите причину неустойчивости PLSA.

В связи с отсутствием ограничений по регуляризации PLSA подвержена переполнению из-за большого количества параметров $\phi_{wd}$ и $\theta_{td}$.

**4)** На каких выборках наиболее заметна разница в работе PLSA и LDA?

На выборках, в которых число тем у каждого из документов мало.

**5)** По каким причинам в ЕМ-алгоритме для тематического моделирования E-шаг встраивается внутрь М-шага?

Матрица $H_{dwt}$ распределения между документами $d$, словами $w$ и темами $t$ вычисляется не целиком, а частично, по мере прохода по набору документов $D$.

**6)** Опишите применение тематического моделирования в задаче информационного поиска.

Тематическое моделирование в задаче информационного поиска можно применять, например, для нахождения документов по заданным темам, анализа тематической структуры документа, выделения основных тем документа.

**7)** В чем основная причина сложности обработки русского языка по сравнению с английским?

* Отсутствие строгого порядка слов в предложениях.
* Наличие различных форм у одного и того же слова (окончания в разных падежах, спряжение глаголов).
* У различных форм одного слова могут быть различные признаки и свойства, поэтому возникают сложности при построении векторного представления слова (одно представление на слово или разные для каждой формы?).
* Наличие омонимов, в том числе представляющих различные части речи, например "село" (сущ.) и "село" (гл.).

**8)** Каким образом парсинг зависимостей между словами помогает в решении задач обработки естественного языка?

* Выделение ключевых слов и темы предложения.
* Разрешение кореференций, то есть сопоставление местоимению существительного, которое оно заменяет.
* Извлечение синтаксических конструкций из текста, что зачастую позволяет разрешить омонимию (пример в 7 вопросе).

**9)** Что такое кореференции?

Кореференция$~-$ связывание нескольких отсылок в документе к одному реальному объекту, например: замена существительного на местоимение, использование синонимов, аббревиатур.

**10)** В чем отличие между CBOW и Skip-gram?

* Суть работы CBOW$~-$ обучение нейронной сети для предсказания слова по его контексту, то есть $p(v|w) = \frac{\exp(B_v^T(\sum_{j}A_j))}{\sum_s\exp(B_s^T(\sum_{j}A_j)))}$.
* Суть работы Skip-gram$~-$ наоборот, обучение нейронной сети для предсказания контекста по данному слову, то есть $p(v|w) = \frac{\exp(B_v^T A_w)}{\sum_s\exp(B_s^T A_w)}$.

# Задачи

## Задача 1 (10%)

#### Посчитайте PageRank для заданного графа вручную и при помощи алгоритма, описанного в семинаре. Результаты сравните.

Сначала пронумеруем вершины: центральную назовём $v_0$, остальные — $v_1 \dots v_9$.

Очевидно, что PageRank вершин $v_1 \dots v_9$ будет совпадать. Обозначим его $p$, а PageRank вершины $v_0$ — $p_0$.

Посчитаем по формуле $p(d) = \dfrac{1-\delta}{N} + \delta\sum_{c \in D^{in}_{d}}\dfrac{PR(c)}{|D^{out}_{c}|}$:

$p_0 = \dfrac{1-0.85}{9} + 0.85*8*\dfrac{p}{3}$

$p = \dfrac{1-0.85}{9} + 0.85*2*\dfrac{p}{3}$

Решив систему и отнормировав, получим: $p_0 \approx 0.252337, p \approx 0.093458$.

Реализуем алгоритм с семинара:

In [1]:
import numpy as np

adj = [[1, 1, 1, 1, 1, 1, 1, 1, 1],
       [1, 0, 1, 0, 0, 0, 0, 0, 1],
       [1, 1, 0, 1, 0, 0, 0, 0, 0],
       [1, 0, 1, 0, 1, 0, 0, 0, 0],
       [1, 0, 0, 1, 0, 1, 0, 0, 0],
       [1, 0, 0, 0, 1, 0, 1, 0, 0],
       [1, 0, 0, 0, 0, 1, 0, 1, 0],
       [1, 0, 0, 0, 0, 0, 1, 0, 1],
       [1, 1, 0, 0, 0, 0, 0, 1, 0]]
adj = np.array(adj)

adj_norm = np.empty((adj.shape[0], adj.shape[0]))

for j in range(len(adj)):
    adj_norm[:,j] = adj[:,j] / adj[:,j].sum()

def pagerank(G, d=0.85, e=1e-4):
    n = G.shape[0]
    v = np.random.rand(n)
    v /= v.sum()
    v_last = np.array([1. for _ in range(n)])
    M = d*G + (1-d)/n * np.array([[1. for _ in range(n)] for _ in range(n)])
    while np.square(v - v_last).sum() > e:
        v_last = v
        v = M.dot(v)
    return v
    
pagerank(adj_norm)

array([0.25233404, 0.09227719, 0.09440902, 0.09279838, 0.09444309,
       0.09266218, 0.09448459, 0.09214099, 0.09445052])

Проверим результат с помощью NetworkX:

In [2]:
import networkx as nx

nx.graph = nx.from_numpy_matrix(adj)
nx.pagerank(nx.graph)

{0: 0.25233621974128034,
 1: 0.09345797253233992,
 2: 0.09345797253233992,
 3: 0.09345797253233992,
 4: 0.09345797253233992,
 5: 0.09345797253233992,
 6: 0.09345797253233992,
 7: 0.09345797253233992,
 8: 0.09345797253233992}

## Задача 2 (10%)

#### Пользователь браузера в дополнение к кликам по ссылкам один раз может перейти по кнопке *Назад* и вернуться на предыдущую страницу. Можно ли такую модель описать с помощью однородной марковской цепи? Если да, опишите, если нет, докажите.

Цепь Маркова называется **однородной**, если матрица переходных вероятностей не зависит от номера шага, то есть:

$P_{ij}(n) = P_{ij}, \forall{n} \in N$

Будем хранить в качестве состояния пару $(p_{cur}, p_{last})$, то есть текущую страницу и предыдущую страницу (если предыдущей нет, то $-1$).

Тогда добавим следующие переходы:
* Если $\exists$ ссылка из $p_1$ в $p_2$: $(*, p_1) \rightarrow (p_1, p_2)$.
* Для перехода назад: $(p_1, p_2) \rightarrow (-1, p_1)$.

## Задача 3 (15%)

#### Опишите вероятностные предположения, на которые опирается TF-IDF при подсчете вероятностей.

#### Пусть задана колекция текстовых документов $d_1, d_2,\ldots, d_n$, состоящая из двух видов слов: $w_1$ и $w_2$. В документе $d_i$ ровно $k_{i1}$ слов $w_1$ и $k_{i2}$ слов $w_2$.

#### Оцените вероятность встретить $k$ раз слово $w_1$. Сравните с оценкой вероятности, используемой в TF-IDF.

#### Совпадают ли эти значения? Если нет, проведите анализ "источника" различий.

Зададим опреления:
* TF-IDF$(q,d)~-$ мера релевантности документа $d$ запросу $q$.
* $n_{dw}~-$ число вхождений слова $w$ в документ $d$.
* $N_w~-$ число документов, содержащих слово $w$.
* $N~-$ общее число документов.
* $\dfrac{N_w}{N}~-$ вероятность встретить слово $w$ в случайном документе из набора.
* $\left(\dfrac{N_w}{N}\right)^{n_{dw}}~-$ вероятность встретить слово $w$ $n_{dw}$ раз.
* $P(q, d) = \prod_{w \in q}{\left(\dfrac{N_w}{N}\right)^{n_{dw}}}~-$ вероятность встретить в документе $d$ слова $w \in q$ из запроса $q$.

Обычная вероятность встретить $k$ раз слово $w_1$ равна $\frac{\sum_i{I(k_{i1}>=k)}}{n}$.
Вероятность в случае TF-IDF равна $\left(\frac{\sum_i{I(k_{i1}>0)}}{n}\right)^{k}$.

Числа будут совпадать, если количество документов, содержащих $k$ раз слово $w_1$ будет убывать экспоненциально быстро с ростом $k$.

Если мы не накладываем никаких дополнительных ограничений, то эти вероятности будут отличаться (чем больше $k$, тем больше будет разница). Вероятности совпадаюи при $k=1$.

Как мы выяснили, оценка TF-IDF будет экспоненциально быстро убывать с ростом $k$, в то время как "обычная" вероятность так быстро убывать не будет.

## Задача 4 (10%)

#### Задано 10 документов. Их отранжировали идеально, а затем 4 и 6 документы поменяли местами. Подсчитайте коэффициент ранговой корреляции ($\tau$ Кенделла).

По определению:

$\tau = \dfrac{P-Q}{C_n^2}$, где:

* $P~-$ количество пар с совпадающим взаимным порядком,
* $Q~-$ количество пар с несовпадающим взаимным порядком,
* $n~-$ количество документов.

Получим:

$\tau = \dfrac{42-3}{45} = 0.8(6) \approx 0.8667$

In [3]:
from scipy.stats import kendalltau

old = list(range(1, 11))
new = old.copy()
new[3], new[5] = new[5], new[3]

print(old, new, sep='\n')

kendalltau(old, new).correlation

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
[1, 2, 3, 6, 5, 4, 7, 8, 9, 10]


0.8666666666666666

## Задача 5 (20%)

#### С какой целью общеупотребительные слова исключают из рассмотрения при построении тематической модели? Если их не исключать, как это отразится на матрицах $\Phi$ и $\Theta$?

* Общеупотребительные слова (стоп-слова) содержатся в большинстве тем в большом количестве и не несут смысловой нагрузки: не позволяют определить ту или иную тему документа.
* Для матрицы $\Phi$. Из-за того, что вероятность встретить стоп-слово в любом тексте с любой темой большая, в строках матрицы будут записаны большие числа. В строках, соответствующих другим словам, будут находится сравнительно маленькие числа, снижая таким образом релевантность тематического поиска.
* Для матрицы $\Theta$. По определению $\theta_{td} = p(t|d)$. При добавлении в рассмотрение общеупотребительных слов ко всем значениям $\theta_{td}$ добавится константа. Таким образом, при дальнейшей нормировке уменьшится разброс чисел и, соответственно, дисперсия. Это ведёт к неустойчивости $EM$-алгоритма.

## Задача 6 (25%)

Задано значение $KL(P||Q)$. Можно ли оценить значение $KL(Q||P)$? Если да, то оцените; если нет, то обоснуйте.

Если $P$ и $Q$ имеют одинаковые распределения, то $KL(P||Q) = KL(Q||P) = 0$.

В противном случае рассмотрим $P: p(0) = p(1) = \dfrac12$, $Q: q(0) = \dfrac1n, q(1) = \dfrac{n-1}{n}$.

Тогда $|KL(P||Q) - KL(Q||P)| = \left|\dfrac{1}{2}\log\left(\dfrac{\frac{1}{2}}{\frac{n-1}{n}}\right) + \dfrac{1}{2}\log\left(\dfrac{\frac{1}{2}}{\frac{1}{n}}\right) - \dfrac{n-1}{n}\log\left(\dfrac{\frac{n-1}{n}}{\frac{1}{2}}\right) + \left(\dfrac{1}{n}\right)\log\left(\dfrac{\frac{1}{n}}{\frac{1}{2}}\right)\right| \ = \ = \left(\dfrac{n+2}{2n}\log\left(\dfrac{n-1}{2}\right) + \dfrac{3n-2}{2n}\log\left(\dfrac{n-1}{2n}\right)\right) \to \infty, n \to \infty$.

Видим, что разность между этими величинами, может как равняться $0$, так и стремиться к бесконечности.

## Задача 7 (бонусная +25%)

#### Рассмотрим пример из семинара:

In [4]:
import spacy
from spacy import displacy
text = """But Google is starting from behind. The company made a late push
into hardware, and Apple’s Siri, available on iPhones, and Amazon’s Alexa
software, which runs on its Echo and Dot devices, have clear leads in
consumer adoption."""
nlp = spacy.load('en_core_web_sm')
doc = nlp(text)
displacy.render(doc, style='ent', jupyter=True)

Сразу заметно, что некорректные метки географических объектов расположены в местах перевода строки. Попробуем заменить их на пробелы.

In [5]:
new_text = text.replace('\n', ' ')
new_doc = nlp(new_text)
displacy.render(new_doc, style='ent', jupyter=True)

Как видим, некорректные геометки пропали.